<a href="https://colab.research.google.com/github/cedricsoares/optimizer_dataset_avec_pandas/blob/master/optimisation_police.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!sudo pip install csvkit

     |████████████████████████████████| 3.8MB 29kB/s 
     |████████████████████████████████| 102kB 29.4MB/s 
     |████████████████████████████████| 163kB 48.1MB/s 
     |████████████████████████████████| 61kB 22.1MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/5f/be/3f/d151aff6c6cce1aa1d56233d68c4b9d38b045bbe5fda018d45
  Stored in directory: /root/.cache/pip/wheels/8a/2f/99/dbf1c6af14192030927240678c0d2176b479dcc44b51a3a6d0
  Stored in directory: /root/.cache/pip/wheels/06/21/5a/84eb0a6b77a3b3d3254b30aaf993533b2236fd29083e120b24
  Stored in directory: /root/.cache/pip/wheels/4c/a3/09/e002ac6fb3921eb0aefd511f7aeca7d0b817b4fdb4b441cc7e
  Stored in directory: /root/.cache/pip/wheels/e9/d0/db/aa6af26d9762852afc0c982d96f9b4f29a373205889453555b
Successfully built csvkit agate-excel agate-dbf agate-sql parsedatetime


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/ny_police.csv")
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,raw_row_number,date,time,location,county_name,subject_age,subject_race,subject_sex,type,violation,speed,posted_speed,vehicle_color,vehicle_make,vehicle_model,vehicle_type,vehicle_registration_state,vehicle_year
0,1,2010-10-05,17:01:00,PALISADES PARKWAY SB|PIPK|9 - PARKWAY,Rockland County,25.0,white,male,vehicular,NO FRONT PLATE DISPLAY,NaN,NaN,gl - gold,chev,NaN,1 - passenger,NY,2000.0
1,2,2010-01-19,14:51:00,ST-16|16|2 - STATE,Cattaraugus County,54.0,white,male,vehicular,EXPIRED INSPECTION 12/31/09,NaN,NaN,wh - white,intl,NaN,6 - truck/tractor,NY,2004.0
2,3,2010-02-12,10:33:00,TAMARAC ROAD S/B|129|3 - COUNTY,Rensselaer County,21.0,white,male,vehicular,UNINSPECTED MOTOR VEHICLE,NaN,NaN,bl - blue,suba,NaN,1 - passenger,NY,2008.0
3,4,2010-04-19,20:36:00,I-87|87|1 - INTERSTATE,Albany County,25.0,white,male,vehicular,UNREGISTERED MOTOR VEHICLE,NaN,NaN,bk - black,toyt,NaN,1 - passenger,NY,2005.0
4,5,2010-07-10,13:53:00,WILDER ST / I-490 E|490|1 - INTERSTATE,Monroe County,69.0,black,male,vehicular,UNINSPECTED MOTOR VEHICLE,NaN,NaN,rd - red,niss,NaN,1 - passenger,NY,2008.0


In [0]:
optimized_df = df.copy()

In [47]:
#On affiche le détail de l'utilisation de la mémoire ainsi que la répartition par colonne des types donnnées.
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7962169 entries, 0 to 7962168
Data columns (total 18 columns):
raw_row_number                int64
date                          object
time                          object
location                      object
county_name                   object
subject_age                   float64
subject_race                  object
subject_sex                   object
type                          object
violation                     object
speed                         float64
posted_speed                  float64
vehicle_color                 object
vehicle_make                  object
vehicle_model                 object
vehicle_type                  object
vehicle_registration_state    object
vehicle_year                  float64
dtypes: float64(4), int64(1), object(13)
memory usage: 6.5 GB


In [48]:
df_obj = df.select_dtypes(include=['object']).copy()
df_obj.describe()

,date,time,location,county_name,subject_race,subject_sex,type,violation,vehicle_color,vehicle_make,vehicle_model,vehicle_type,vehicle_registration_state
count,7962169,7962169,7074742,7962157,7962150,7960212,7962169,7962146,7907739,7950304,93,7962131,7722895
unique,2905,1440,400047,63,5,2,1,340622,15,5782,47,14,51
top,2010-05-25,11:00:00,I-87|87|1 - INTERSTATE,Erie County,white,male,vehicular,SPEED IN ZONE,gy - grey,chev,savana,1 - passenger,NY
freq,5410,17961,149638,469716,5926237,5454829,7962169,1524591,2056972,1017588,6,6789930,6296592


In [50]:
df_obj.groupby('subject_sex').size()

subject_sex
female    2505383
male      5454829
dtype: int64

In [0]:
converted_obj = pd.DataFrame()

for col in df_obj.columns:
    if len(df_obj[col].unique()) / len(df_obj[col]) < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [0]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)    
    usage_mb = usage_b / 1024 ** 2 # convertir les bytes en megabytes
    return "{:03.2f} MB".format(usage_mb) # afficher sous format nombre (min 3 chiffres) et une précision de deux décimales.

In [60]:
print(mem_usage(df_obj))
print(mem_usage(converted_obj))

6375.61 MB
252.79 MB


In [0]:
df_int = df.select_dtypes(include=['int']).copy()

In [61]:
df_int.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7962169 entries, 0 to 7962168
Data columns (total 1 columns):
raw_row_number    int64
dtypes: int64(1)
memory usage: 60.7 MB


In [0]:
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')

In [59]:
converted_int.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7962169 entries, 0 to 7962168
Data columns (total 1 columns):
raw_row_number    uint32
dtypes: uint32(1)
memory usage: 30.4 MB


In [62]:
print(mem_usage(df_int))
print(mem_usage(converted_int))

60.75 MB
30.37 MB


In [63]:
df_float = df.select_dtypes(include=['float']).copy()
df_float.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7962169 entries, 0 to 7962168
Data columns (total 4 columns):
subject_age     float64
speed           float64
posted_speed    float64
vehicle_year    float64
dtypes: float64(4)
memory usage: 243.0 MB


In [0]:
converted_float = df_float.apply(pd.to_numeric,downcast='float')

In [65]:
converted_float.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7962169 entries, 0 to 7962168
Data columns (total 4 columns):
subject_age     float32
speed           float32
posted_speed    float32
vehicle_year    float32
dtypes: float32(4)
memory usage: 121.5 MB


In [66]:
print(mem_usage(df_float))
print(mem_usage(converted_float))

242.99 MB
121.49 MB


In [0]:
optimized_df[converted_obj.columns] = converted_obj
optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float


In [68]:
print(mem_usage(df))
print(mem_usage(optimized_df))

6679.35 MB
404.66 MB


In [70]:
dtypes = optimized_df.drop(['date', 'time'],axis=1).dtypes
column_types = dict(zip(dtypes.index, [i.name for i in dtypes.values]))
print(column_types)

{'raw_row_number': 'uint32', 'location': 'category', 'county_name': 'category', 'subject_age': 'float32', 'subject_race': 'category', 'subject_sex': 'category', 'type': 'category', 'violation': 'category', 'speed': 'float32', 'posted_speed': 'float32', 'vehicle_color': 'category', 'vehicle_make': 'category', 'vehicle_model': 'category', 'vehicle_type': 'category', 'vehicle_registration_state': 'category', 'vehicle_year': 'float32'}


In [71]:
%%time #Pour mesurer le temps d'éxécution de la cellule 
read_and_optimized = pd.read_csv('/content/drive/My Drive/ny_police.csv',dtype=column_types,parse_dates=[['date', 'time']],infer_datetime_format=True)
print(mem_usage(read_and_optimized))
read_and_optimized.head()

434.64 MB
CPU times: user 50.5 s, sys: 2.53 s, total: 53 s
Wall time: 53.6 s


In [72]:
read_and_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7962169 entries, 0 to 7962168
Data columns (total 17 columns):
date_time                     datetime64[ns]
raw_row_number                uint32
location                      category
county_name                   category
subject_age                   float32
subject_race                  category
subject_sex                   category
type                          category
violation                     category
speed                         float32
posted_speed                  float32
vehicle_color                 category
vehicle_make                  category
vehicle_model                 category
vehicle_type                  category
vehicle_registration_state    category
vehicle_year                  float32
dtypes: category(11), datetime64[ns](1), float32(4), uint32(1)
memory usage: 434.6 MB


In [0]:
from pathlib import Path
path = Path("/content/drive/My Drive")

In [0]:
read_and_optimized.to_hdf(path/'ny_police.h5', key='read_and_optimized', format="table")

In [76]:
%%time
df_test = pd.read_hdf(path/'ny_police.h5')

CPU times: user 5.63 s, sys: 1.16 s, total: 6.79 s
Wall time: 7.32 s


In [77]:
%%time
df_test.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7962169 entries, 0 to 7962168
Data columns (total 17 columns):
date_time                     datetime64[ns]
raw_row_number                uint32
location                      category
county_name                   category
subject_age                   float32
subject_race                  category
subject_sex                   category
type                          category
violation                     category
speed                         float32
posted_speed                  float32
vehicle_color                 category
vehicle_make                  category
vehicle_model                 category
vehicle_type                  category
vehicle_registration_state    category
vehicle_year                  float32
dtypes: category(11), datetime64[ns](1), float32(4), uint32(1)
memory usage: 495.4 MB
CPU times: user 516 ms, sys: 983 µs, total: 517 ms
Wall time: 517 ms
